In [ ]:
# Follow detectron2 setup
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Setup packages
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, sys
from google.colab.patches import cv2_imshow
from google.colab import drive

from pycocotools.coco import COCO
import requests

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer

In [ ]:
# Mount drive
drive.mount('/content/drive')

# Set path to downloaded COCO API files
os.chdir("/content/drive/My Drive/Colab Notebooks/CS230Project/OpenImages/cocoapi-master/PythonAPI")
sys.path.append("/content/drive/My Drive/Colab Notebooks/CS230Project/OpenImages/cocoapi-master/PythonAPI")

# Install downloaded COCO API
!python3 setup.py install
!pip install awscli

In [ ]:
# Reset path for annotation conversion
os.chdir("/content/drive/My Drive/Colab Notebooks/CS230Project/COCO")
sys.path.append("/content/drive/My Drive/Colab Notebooks/CS230Project/COCO")


In [ ]:
# Implement convert_annotations.py from openimages2coco github repository, courtesy of the Bethge Lab
!python3 convert_annotations.py -p "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO" --subsets val

In [ ]:
# Tidy up annotation json to only include desired classes

# Filter out so only "bottle" remains in train annotation file
python3 filter.py --input_json "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_train2017.json" --output_json "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_train2017_filtered.json"  --categories bottle

# Filter out so only "bottle" remains in "val" annotation file (used as development)
python3 filter.py --input_json "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_val2017.json" --output_json "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_val2017_filtered.json"  --categories bottle

In [ ]:
# Selectively download just the bottle images to my drive

# instantiate COCO specifying the annotations json path
coco = COCO("/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_train2017_filtered.json")
# Specify a list of category names of interest
catIds = coco.getCatIds(catNms=['bottle'])
# Get the corresponding image ids and images using loadImgs
imgIds = coco.getImgIds(catIds=catIds)
images = coco.loadImgs(imgIds)

# Save the images into a local folder (up to 8501 in increments to avoid Google Colab timeout)
for im in images[7500:8501]:
    img_data = requests.get(im['coco_url']).content
    with open("/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/ims/train/" + im['file_name'], 'wb') as handler:
        handler.write(img_data)

In [ ]:
# Register datasets into detectron2 by calling the annotation file and image path as follows:
# register_coco_instances("YourTestDatasetName", {}, "path to test.json", "path to test image folder")

register_coco_instances("justBottleCOCOTrain", {},"/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_train2017_filtered.json", "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/ims/train")

register_coco_instances("justBottleCOCOVal", {},"/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_val2017_filtered.json", "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/ims/val")

register_coco_instances("COCOVal", {},"/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/anns/instances_val2017.json", "/content/drive/My Drive/Colab Notebooks/CS230Project/COCO/ims/val")


# Establish dicts and metadata for visualization later
bottleData_dicts_Train = DatasetCatalog.get("justBottleCOCOTrain")
bottle_metadata_Train = MetadataCatalog.get("justBottleCOCOTrain")

bottleData_dicts_Val = DatasetCatalog.get("justBottleCOCOVal")
bottle_metadata_Val = MetadataCatalog.get("justBottleCOCOVal")

data_dicts_Val = DatasetCatalog.get("COCOVal")
metadata_Val = MetadataCatalog.get("COCOVal")


In [ ]:
# Ensure visualization and dataset registration was successful by displaying random images and corresponding annotations
for d in random.sample(bottleData_dicts_Val, 3):
    # print(d)
    fn = d["file_name"]
    img = cv2.imread(fn)
    visualizer = Visualizer(img[:, :, ::-1], metadata=bottle_metadata_Val, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
# Set up the pretrained configuration for baseline and future training
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("justBottleCOCOTrain",)
cfg.DATASETS.TEST = ("justBottleCOCOVal",)
# cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo





In [ ]:
# Read deployment images for qualitative analysis
im1 = cv2.imread('/content/drive/My Drive/Colab Notebooks/CS230Project/rainsCabinetSmall.jpg')
im2 = cv2.imread('/content/drive/My Drive/Colab Notebooks/CS230Project/harveyCabinetSmall.jpg')

In [ ]:
# Establish threshold and pretrained predictor
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor_init = DefaultPredictor(cfg)

# Display predictor's ability to perform inference on "bottle" dataset
for d in random.sample(bottleData_dicts_Val, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor_init(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_Val, 
                   scale=0.5, 
                  #  instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Establish training configuration
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (bottle)
cfg.TEST.DETECTIONS_PER_IMAGE = 100 # Limit total number of detections to 50

cfg.SOLVER.IMS_PER_BATCH = 2 # Set batch size
cfg.SOLVER.BASE_LR = 0.001  # Pick a LR
cfg.SOLVER.MOMENTUM = 0.95 # Pick beta for momentum
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy problem
cfg.SOLVER.STEPS = [] #(200,)        # either do or dont decay learning rate at step 200
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster, and good enough for this toy problem (default: 512)


In [ ]:
# Train

cfg.SOLVER.CHECKPOINT_PERIOD = 50 # Make sure we can resume if training fails for any reason

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
resume_dir = os.getcwd()+'/output/'
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) #Try "resume_dir" here if wanting to resume; "resume=False" if starting from scratch
trainer.train()


In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
predictor = DefaultPredictor(cfg)

In [ ]:
# Visualize new performance on random examples
for d in random.sample(bottleData_dicts_Val, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=bottle_metadata_Val, 
                   scale=0.5, 
                  #  instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # reset threshold if needed
 
# Using the pretrained model, perform inference on deployment images and visualize

# Example 1
output1 = predictor_init(im1)

# Use `Visualizer` to draw the predictions on the image.
v = Visualizer(im1[:, :, ::-1], metadata_Val, scale=1.2) # MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
out = v.draw_instance_predictions(output1["instances"].to("cpu"))

# Display and save the annotated image
out_im = out.get_image()[:, :, ::-1]
cv2_imshow(out_im)

# Example 2
output2 = predictor_init(im2)

# Use `Visualizer` to draw the predictions on the image.
v = Visualizer(im2[:, :, ::-1], metadata_Val, scale=1.2) # MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
out = v.draw_instance_predictions(output2["instances"].to("cpu"))

# Display and save the annotated image
out_im = out.get_image()[:, :, ::-1]
cv2_imshow(out_im)



# Using the model after transfer learning, perform inference on deployment images and visualize

# Example 1
output1 = predictor(im1)

# Use `Visualizer` to draw the predictions on the image.
v = Visualizer(im1[:, :, ::-1], bottle_metadata_Val, scale=1.2) # MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
out = v.draw_instance_predictions(output1["instances"].to("cpu"))

# Display and save the annotated image
out_im = out.get_image()[:, :, ::-1]
cv2_imshow(out_im)

# Example 2
output2 = predictor(im2)

# Use `Visualizer` to draw the predictions on the image.
v = Visualizer(im2[:, :, ::-1], bottle_metadata_Val, scale=1.2) # MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
out = v.draw_instance_predictions(output2["instances"].to("cpu"))

# Display and save the annotated image
out_im = out.get_image()[:, :, ::-1]
cv2_imshow(out_im)

In [ ]:
# Evaluate the configuration to obtain an AP

evaluator = COCOEvaluator("justBottleCOCOVal", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "justBottleCOCOVal")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`